In [7]:
import pandas as pd

df = pd.read_csv('..\data\data.csv')
print(df.columns.tolist())

['data_donor_email', 'session_id', 'version', 'event_time', 'A_KEY_TIME_MEAN', 'BOTH_DEV_CC_COUNT', 'BOTH_DEV_C_COUNT', 'BOTH_DEV_M_COUNT', 'CLUSTERED_KEYS', 'CLUSTERED_NR_KEYS', 'CLUSTERED_R_KEYS', 'DELETE_KEY', 'DELETE_KEYS', 'GAP_D1', 'GAP_D2', 'GAP_D3', 'GAP_D4', 'GAP_D5', 'GAP_D6', 'GAP_D7', 'GAP_D8', 'GAP_D9', 'GAP_SD', 'KEY_COUNT', 'KEY_GAP_TIME_D1', 'KEY_GAP_TIME_D2', 'KEY_GAP_TIME_D3', 'KEY_GAP_TIME_D4', 'KEY_GAP_TIME_D5', 'KEY_GAP_TIME_D6', 'KEY_GAP_TIME_D7', 'KEY_GAP_TIME_D8', 'KEY_GAP_TIME_D9', 'KEY_TIME', 'KEY_TIME_D1', 'KEY_TIME_D2', 'KEY_TIME_D3', 'KEY_TIME_D4', 'KEY_TIME_D5', 'KEY_TIME_D6', 'KEY_TIME_D7', 'KEY_TIME_D8', 'KEY_TIME_D9', 'KEY_TIME_MEAN', 'MANY_KEYS_BY_KEY_PRESSED', 'MOUSE_CLICK_RATIO', 'MOUSE_COUNT', 'MOUSE_DDIR_D1', 'MOUSE_DDIR_D2', 'MOUSE_DDIR_D3', 'MOUSE_DDIR_D4', 'MOUSE_DDIR_D5', 'MOUSE_DDIR_D6', 'MOUSE_DDIR_D7', 'MOUSE_DDIR_D8', 'MOUSE_DIR_D1', 'MOUSE_DIR_D2', 'MOUSE_DIR_D3', 'MOUSE_DIR_D4', 'MOUSE_DIR_D5', 'MOUSE_DIR_D6', 'MOUSE_DIR_D7', 'MOUSE_DIR_D

In [13]:
import time

import pandas as pd
from pandas import DataFrame
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, BernoulliNB, ComplementNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from tqdm import tqdm


def remove_na_values(df: DataFrame, drop_column_threshold=0.15):
    null_var = df.isnull().sum() / df.shape[0]
    drop_columns = null_var[null_var > drop_column_threshold].keys()
    df = df.drop(columns=drop_columns)
    df = df.dropna()
    return df


def train_models(models: list, df: DataFrame):
    results = pd.DataFrame(columns=['model', 'acc', 'tpr'])
    for model in tqdm(models):
        x = df.iloc[:, 4:]
        y = df.iloc[:, 0]

        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

        model.fit(x_train, y_train)
        # y_pred_proba = model.predict_proba(x_test)
        y_pred = model.predict(x_test)

        # auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovo')
        acc = accuracy_score(y_test, y_pred)
        tpr = recall_score(y_test, y_pred, average='micro')

        results = results.append({'model': str(model), 'acc': acc, 'tpr': tpr}, ignore_index=True)
    return results, models


In [81]:
import re

df = remove_na_values(pd.read_csv('C:\\Users\\smatuszewski_adm\\Desktop\\PW\\4sem\\DSW\\DataScienceWorkshop2023L\\data\\data.csv'))

column_names = df.columns[4:]

def get_group(s):
    pattern = re.compile(r"(.+?)_D[0-9]")
    search = pattern.search(s)
    if search is None:
        return s
    return search.group(1)

feature_groups = np.array([get_group(s) for s in column_names])

In [69]:
def remove_columns_ints(df: DataFrame, exclude_columns: list):
    columns_to_remove = [x + 4 for x in exclude_columns]
    print(columns_to_remove)
    print(exclude_columns)
    result = df.drop(df.columns[columns_to_remove], axis=1)
    return result

In [95]:
def remain_columns_ints(df: DataFrame, remain_columns: list):
    columns_to_remain = [0,1,2,3] + [x + 4 for x in remain_columns]
    
    result = df.iloc[:, columns_to_remain]
    print(result.shape)
    return result

In [16]:
feature_groups

array(['A_KEY_TIME_MEAN', 'BOTH_DEV_C_COUNT', 'GAP', 'GAP', 'GAP', 'GAP',
       'GAP', 'GAP', 'GAP', 'GAP', 'GAP', 'GAP_SD', 'KEY_COUNT',
       'KEY_GAP_TIME', 'KEY_GAP_TIME', 'KEY_GAP_TIME', 'KEY_GAP_TIME',
       'KEY_GAP_TIME', 'KEY_GAP_TIME', 'KEY_GAP_TIME', 'KEY_GAP_TIME',
       'KEY_GAP_TIME', 'KEY_TIME', 'KEY_TIME', 'KEY_TIME', 'KEY_TIME',
       'KEY_TIME', 'KEY_TIME', 'KEY_TIME', 'KEY_TIME', 'KEY_TIME',
       'KEY_TIME', 'KEY_TIME_MEAN', 'MOUSE_COUNT', 'MOUSE_DDIR',
       'MOUSE_DDIR', 'MOUSE_DIR', 'MOUSE_DIR', 'MOUSE_DIR', 'MOUSE_DIR',
       'MOUSE_DIR', 'MOUSE_DIR', 'MOUSE_DIR', 'MOUSE_DIR', 'MOUSE_DIR',
       'MOUSE_DIST', 'MOUSE_DIST', 'MOUSE_DIST', 'MOUSE_DIST',
       'MOUSE_DIST', 'MOUSE_DIST', 'MOUSE_DIST', 'MOUSE_DIST',
       'MOUSE_DIST', 'MOUSE_GAP', 'MOUSE_GAP', 'MOUSE_GAP', 'MOUSE_GAP',
       'MOUSE_GAP', 'MOUSE_GAP', 'MOUSE_GAP', 'MOUSE_GAP', 'MOUSE_GAP',
       'MOUSE_SPEED', 'MOUSE_SPEED', 'MOUSE_SPEED', 'MOUSE_SPEED',
       'MOUSE_SPEED', 'MOUSE_SPEE

In [17]:
feature_groups_map = {s: np.where(feature_groups == s)[0] for s in np.unique(feature_groups)}

In [31]:
feature_groups_map

{'A_KEY_TIME_MEAN': array([0], dtype=int64),
 'BOTH_DEV_C_COUNT': array([1], dtype=int64),
 'GAP': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64),
 'GAP_SD': array([11], dtype=int64),
 'KEY_COUNT': array([12], dtype=int64),
 'KEY_GAP_TIME': array([13, 14, 15, 16, 17, 18, 19, 20, 21], dtype=int64),
 'KEY_TIME': array([22, 23, 24, 25, 26, 27, 28, 29, 30, 31], dtype=int64),
 'KEY_TIME_MEAN': array([32], dtype=int64),
 'MOUSE_COUNT': array([33], dtype=int64),
 'MOUSE_DDIR': array([34, 35], dtype=int64),
 'MOUSE_DIR': array([36, 37, 38, 39, 40, 41, 42, 43, 44], dtype=int64),
 'MOUSE_DIST': array([45, 46, 47, 48, 49, 50, 51, 52, 53], dtype=int64),
 'MOUSE_GAP': array([54, 55, 56, 57, 58, 59, 60, 61, 62], dtype=int64),
 'MOUSE_SPEED': array([63, 64, 65, 66, 67, 68, 69, 70, 71], dtype=int64),
 'MULTIPLE_KEYS_DOWN': array([72, 73, 74, 75, 76, 77, 78, 79, 80, 81], dtype=int64),
 'MULTIPLE_KEYS_UP': array([82, 83, 84, 85, 86, 87, 88, 89, 90, 91], dtype=int64)}

In [32]:
feature_groups_map['GAP']

array([ 2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=int64)

In [ ]:
models = [
    # KNeighborsClassifier(3, weights="distance"),
    # GaussianNB(),
    # BernoulliNB(),
    # ComplementNB(),
    # MultinomialNB(),
    RandomForestClassifier(),
    # LogisticRegression(),
    # LinearDiscriminantAnalysis(),
    # MLPClassifier(),
    # GradientBoostingClassifier() #likely to require approx. 30min of training time
]

# df = remove_na_values(pd.read_csv('..\data\data.csv'))

results_df_groups = pd.DataFrame(columns=['removed_group', 'n_columns', 'model', 'acc', 'tpr'])

for key, value in feature_groups_map.items():

    print(key)
    print("i:", value)

    df_trimmed = remove_columns_ints(df, value)

    results, models = train_models(models, df_trimmed)

    print(type(results['acc']), results['acc'][0])
    results_df_groups = results_df_groups.append({'removed_group': key, 'n_columns': df_trimmed.shape[1] - 4, 'model': results['model'][0], 'acc': results['acc'][0], 'tpr': results['tpr'][0]}, ignore_index=True)

    #results.to_csv(f'classification_results\\{int(time.time())}_{key}.csv', index=False)


In [91]:
results_df_groups


,removed_group,model,acc,tpr
0,A_KEY_TIME_MEAN,RandomForestClassifier(),0.999411,0.999411
1,BOTH_DEV_C_COUNT,RandomForestClassifier(),0.999117,0.999117
2,GAP,RandomForestClassifier(),0.999411,0.999411
3,GAP_SD,RandomForestClassifier(),0.998822,0.998822
4,KEY_COUNT,RandomForestClassifier(),0.998822,0.998822
5,KEY_GAP_TIME,RandomForestClassifier(),0.999117,0.999117
6,KEY_TIME,RandomForestClassifier(),0.999117,0.999117
7,KEY_TIME_MEAN,RandomForestClassifier(),0.998822,0.998822
8,MOUSE_COUNT,RandomForestClassifier(),0.998528,0.998528
9,MOUSE_DDIR,RandomForestClassifier(),0.998528,0.998528


In [92]:
results_df_groups.to_csv(f'classification_results\\groups_removal.csv', index=False)

## Remaining only single groups

In [96]:
models = [
    # KNeighborsClassifier(3, weights="distance"),
    # GaussianNB(),
    # BernoulliNB(),
    # ComplementNB(),
    # MultinomialNB(),
    RandomForestClassifier(),
    # LogisticRegression(),
    # LinearDiscriminantAnalysis(),
    # MLPClassifier(),
    # GradientBoostingClassifier() #likely to require approx. 30min of training time
]

# df = remove_na_values(pd.read_csv('..\data\data.csv'))

results_df_groups_remain = pd.DataFrame(columns=['removed_group', 'n_columns','model', 'acc', 'tpr'])

for key, value in feature_groups_map.items():

    print(key)
    print("i:", value)

    df_trimmed = remain_columns_ints(df, value)

    results, models = train_models(models, df_trimmed)

    print(type(results['acc']), results['acc'][0])
    results_df_groups_remain = results_df_groups_remain.append({'removed_group': key, 'n_columns': df_trimmed.shape[1] - 4,  'model': results['model'][0], 'acc': results['acc'][0], 'tpr': results['tpr'][0]}, ignore_index=True)


A_KEY_TIME_MEAN
i: [0]
(16985, 5)


100%|██████████| 1/1 [00:01<00:00,  1.51s/it]


<class 'pandas.core.series.Series'> 0.221371798645864
BOTH_DEV_C_COUNT
i: [1]
(16985, 5)


100%|██████████| 1/1 [00:01<00:00,  1.92s/it]


<class 'pandas.core.series.Series'> 0.10538710627023845
GAP
i: [ 2  3  4  5  6  7  8  9 10]
(16985, 13)


...

<class 'pandas.core.series.Series'> 0.9626140712393289
MULTIPLE_KEYS_DOWN
i: [72 73 74 75 76 77 78 79 80 81]
(16985, 14)


100%|██████████| 1/1 [00:01<00:00,  1.61s/it]


<class 'pandas.core.series.Series'> 0.24021195172210774
MULTIPLE_KEYS_UP
i: [82 83 84 85 86 87 88 89 90 91]
(16985, 14)


100%|██████████| 1/1 [00:01<00:00,  1.19s/it]

<class 'pandas.core.series.Series'> 0.158375036797174


In [97]:
results_df_groups_remain

,removed_group,model,acc,tpr
0,A_KEY_TIME_MEAN,RandomForestClassifier(),0.221372,0.221372
1,BOTH_DEV_C_COUNT,RandomForestClassifier(),0.105387,0.105387
2,GAP,RandomForestClassifier(),0.989402,0.989402
3,GAP_SD,RandomForestClassifier(),0.061230,0.061230
4,KEY_COUNT,RandomForestClassifier(),0.013247,0.013247
5,KEY_GAP_TIME,RandomForestClassifier(),0.983809,0.983809
6,KEY_TIME,RandomForestClassifier(),0.957021,0.957021
7,KEY_TIME_MEAN,RandomForestClassifier(),0.199588,0.199588
8,MOUSE_COUNT,RandomForestClassifier(),0.025905,0.025905
9,MOUSE_DDIR,RandomForestClassifier(),0.591993,0.591993
